In [1]:
import time
from pyspark.sql import SQLContext 
import pandas as pd 
sqlc=SQLContext(sc)
import pyspark.sql.functions as F

# 1. Duplicate Transactions

In [ ]:
/* Duplicate Transactions

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE duplicate_transactions (
	transaction_id VARCHAR,
	timestamp INTEGER,
	price INTEGER,
	department VARCHAR 
); 


COPY duplicate_transactions
FROM '<Path to product_sql folder>/product_sql/duplicate_transactions/duplicate_transactions.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Medium
Duration: 30 Minutes

Problem:

The duplicate_transactions table contains transaction_id, timestamp, price and department.

Address these three questions:

#1. How many duplicate records are there? For instance, if Row 1, and Row 2 and Row 3 contain
the same values, then there are two duplicate records.

#2. How many unique records have duplications?

#3. Remove duplicate records, only preserving the unique records.

#4. Which department has the highest duplicate records? Return the department
name and count of duplicate records. Assume the possibility that multiple departments
could have the same highest count.

*/

-- Solution #1

-- Sum to get the total number of duplicate records
SELECT SUM(number_of_dups) AS number_of_duplications
FROM (
	-- Use GROUP BY to count repeating records per unique record
	SELECT transaction_id, timestamp, price, department, 
		COUNT(*) - 1 AS number_of_dups
	FROM duplicate_transactions
	GROUP BY transaction_id, timestamp, price, department 
) t;


-- Solution #2

-- Unique records with no duplications would have 0 in number_of_dups;
-- Remove such records then count the rest which are the unique records
-- with duplicate entries.
SELECT COUNT(*)
FROM (
	-- Use GROUP BY to count repeating records per unique record
	SELECT transaction_id, timestamp, price, department, 
		COUNT(*) - 1 AS number_of_dups
	FROM duplicate_transactions
	GROUP BY transaction_id, timestamp, price, department 
) t
WHERE number_of_dups > 0;


-- Solution #3

-- Just apply DISTINCT to remove duplicate entries 
SELECT DISTINCT *
FROM duplicate_transactions;


-- Solution #4

WITH CountDuplicates AS (
	-- Use GROUP BY to count repeating records per unique record
	SELECT transaction_id, timestamp, price, department, 
		COUNT(*) - 1 AS number_of_dups
	FROM duplicate_transactions
	GROUP BY transaction_id, timestamp, price, department 
),
CountDuplicatesPerDept AS (
	-- Sum the duplicate entries per department
	SELECT
		department,
		SUM(number_of_dups) AS number_of_dups
	FROM CountDuplicates
	GROUP BY department
),
RankByDups AS (
	SELECT
		-- Use DENSE_RANK to rank the department given the number of 
		-- duplicate entries. DENSE_RANK considers repeated values.
		department,
		number_of_dups,
		DENSE_RANK() OVER(ORDER BY number_of_dups DESC) AS dups_ranking
	FROM CountDuplicatesPerDept
)
-- Filter on dups_ranking = 1 as that indicates the department(s) with the 
-- highest number of duplications.
SELECT department, number_of_dups  
FROM RankByDups
WHERE dups_ranking = 1;

# 2. Facebook connections

In [ ]:

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE friends_connections (
	date VARCHAR,
	user_id FLOAT,
	receiver_id INTEGER,
	action VARCHAR 
); 


COPY friends_connections
FROM '<Path to product_sql folder>/product_sql/friends_connections/friends_connections.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Medium
Duration: 30 Minutes

Problem:

Facebook’s analytics team wants to understand how users stay connected among friends on their platform. 
The team believes that understanding patterns could help improve an algorithm that matches potential friends. 
Use the friends table below to address questions below. A user can perform the following sequence of actions: 
(1) request or receive, (2) connect, and (3) block.

Address these two questions:

#1. ​Return a list of users who blocked another user after connecting for at least 90 days. 
Show the user_id and receiver_id.

#2. ​For each user, what is the proportion of each action? Note that the receiver_id can 
appear in multiple actions per user, only regard the latest status when calculating 
the distribution.

*/


-- #1 Return a list of users who blocked another user after connecting for at least 90 days. 
-- 	 Show user_id and receiver_id. 

-- Return a pair of users who once connected for at least 90 days then blocked.
SELECT u.user_id, 
       o.receiver_id
FROM
(
	-- Before a user can block another user, the users must be connected. Create two
	-- sub-queries - one with blocked events and another with connected events.
	-- Use inner join on user_ids and receiver_ids.
	(SELECT * 
		FROM connections
		WHERE action = 'Blocked') u
	JOIN
	(SELECT * 
		FROM connections
		WHERE action = 'Connected') o
	ON u.user_id = o.user_id AND u.receiver_id = o.receiver_id
)
WHERE (u.dates - o.dates) >= 90;

-- #2 - For each user, what is the proportion of each action? Note that the receiver_id 
-- can appear in multiple actions per user, only regard the latest action performed 
-- when calculating the distribution.

-- Assign event order by user_id and receiver_id to filter on the latest event
WITH friendship_status AS (
	SELECT *,  
		   ROW_NUMBER() OVER(PARTITION BY user_id, receiver_id ORDER BY dates DESC) AS event_order
	FROM connections 
),  
-- Filter on the latest event per user_id and receiver_id pair
latest_friendship_status AS (
	SELECT *
	FROM friendship_status
	WHERE event_order = 1
),
-- Create a dummy variable column per action type
status_dummy_variables AS (
	SELECT *,
		   CASE WHEN action = 'Sent' THEN 1 ELSE 0 END AS sent,
		   CASE WHEN action = 'Received' THEN 1 ELSE 0 END AS received,
		   CASE WHEN action = 'Connected' THEN 1 ELSE 0 END AS connected,
		   CASE WHEN action = 'Blocked' THEN 1 ELSE 0 END AS blocked
	FROM latest_friendship_status
),
-- For each action column, divide by event order to get proportion of action types per user.
distribution AS (
	SELECT user_id,
		 	SUM(sent) / SUM(event_order) AS sent,
		 	SUM(received) / SUM(event_order) AS received,
		 	SUM(connected) / SUM(event_order) AS connected,
		 	SUM(blocked) / SUM(event_order) AS blocked
	FROM status_dummy_variables
	GROUP BY user_id
)
SELECT * FROM distribution;


# 3. Find Median

In [ ]:
/* Find Median

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE median_users (
	user_id INTEGER,
	user_creation_date DATE
); 

CREATE TABLE median_transactions (
	user_id INTEGER,
	transaction_date DATE,
	transaction_amount INTEGER
);

COPY median_users
FROM '<Path to product_sql folder>/product_sql/find_median/median_users.csv' 
DELIMITER ',' 
CSV HEADER;

COPY median_transactions
FROM '<Path to product_sql folder>/product_sql/find_median/median_transactions.csv' 
DELIMITER ',' 
CSV HEADER;


/*

Difficulty: Hard
Duration: 15 Minutes

Problem:

There are two tables users and transactions. The median_users table contains the user_id 
and user_creation_date. The median_transactions table contains the user_id, transaction_date,
and transaction_amount. A user can purchase as a visitor even before creating an account, 
and they user_id remains the same. 

#1. Among transactions that occurred on or after the date of sign-up, find the median and average
per user.

*/

-- Solution
WITH UserTransactions AS ( 
	-- Join users and transactions table and filter on transactions that came
	-- after the sign-up
	SELECT 
		median_users.user_id,
		user_creation_date,
		transaction_date,
		transaction_amount		
	FROM median_users 
	JOIN median_transactions
		USING (user_id)
	WHERE transaction_date >= user_creation_date
), 
RowNumber AS (
	-- Create row_numbers in ascending and descending orders which will be useful for 
	-- calculating the median. 
	SELECT 
		user_id,
		transaction_date,
		transaction_amount,
		row_number() OVER(PARTITION BY user_id ORDER BY transaction_amount ASC) AS ascending,
		row_number() OVER(PARTITION BY user_id ORDER BY transaction_amount DESC) AS descending
	FROM UserTransactions
)
-- Calculating the average is straightforward as you can use the AVG() function. The median,
-- however, can be somewhat tricky. The descending column serves as a reference point for the 
-- ascending column. When values in the ascending column are within the bounds of descending - 1
-- and ascending + 1, preserve the values. Otherwise null. Averaging this column returns the median.
SELECT user_id,
	   AVG(transaction_amount) AS average,
	   AVG(CASE 
	   		WHEN ascending BETWEEN descending - 1 AND descending + 1 
	   		THEN transaction_amount ELSE NULL 
	   		END) AS median
FROM RowNumber
GROUP BY user_id; 

# 4. Google Ads Spending

In [ ]:
/* Google Ads Spending

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE google_ads_spending (
	date DATE,
	business VARCHAR,
	spending FLOAT
); 


COPY google_ads_spending
FROM '<Path to product_sql folder>/product_sql/google_ads_spending/google_ads_spending.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Medium
Duration: 30 Minutes

Problem:

A social network platform allows businesses to publish advertisements. The platform tracks 
daily advertisment spendings across business accounts. Currently, the platform
doesn’t have a backend system to alert unsual spendings. Address the SQL questions below:

Address these two questions:

#1​.​ Compute the 30-day moving average of advertisement spending per business.

#2​.​ Compute the 30-day moving standard deviation of advertisement spending per business.

#3.​ The platform wants to track anomalous spendings. Create a new column called “outlier” 
which flags any spending that is above or below the two standard deviation from the mean. 
Use the moving average and standard deviation computed in previous steps.

*/


-- #1 For each business, compute the moving average of the costs for the past thirty days.

SELECT *,
	   -- Compute the moving average of spending
	   (SELECT AVG(s2.spending) FROM ads_spending s2 
	   	WHERE (s1.date - s2.date) <= 30 AND
	   		  (s1.date - s2.date) >= 0 AND
	   		   s1.business = s2.business) AS moving_avg
FROM ads_spending s1;

-- #2 - For each business, compute the moving standard deviation of the costs for the past thirty days.

SELECT *,
	   -- Compute the moving std of spending
	   (SELECT POWER(AVG(POWER(s3.spending - t1.moving_avg, 2)), 0.5) FROM ads_spending s3
	   	WHERE (t1.date - s3.date) <= 30 AND
	   		  (t1.date - s3.date) >= 0 AND
	   		   t1.business = s3.business) AS moving_std
FROM (	   		   
	SELECT *,
		   -- Compute the moving average of spending
		   (SELECT AVG(s2.spending) FROM ads_spending s2 
		   	WHERE (s1.date - s2.date) <= 30 AND
				  (s1.date - s2.date) >= 0 AND
		   		   s1.business = s2.business) AS moving_avg
	FROM ads_spending s1
) t1;

-- #3 - The platform wants to track anomalous spendings. Create a new column called “outlier” 
-- which flags any spending that is above or below the two standard deviation from the mean. 
-- Use the moving average and standard deviation computed in previous steps.

SELECT *,
       -- Append a new column called ‘outlier’ that flags outlier spendings
	   (CASE WHEN spending > moving_avg + 2 * moving_std THEN 1
	   		 WHEN spending < moving_avg - 2 * moving_std THEN 1
	   		 else 0
	   	END) AS outlier
FROM (	   		 
	SELECT *,
           -- Compute the moving std of spending
		   (SELECT POWER(AVG(POWER(s3.spending - t1.moving_avg, 2)), 0.5) FROM ads_spending s3
		   	WHERE (t1.date - s3.date) <= 30 AND
		   		  (t1.date - s3.date) >= 0 AND
		   		   t1.business = s3.business) AS moving_std
	FROM (	   		   
		SELECT *,
         	   -- Compute the moving average of spending
			   (SELECT AVG(s2.spending) FROM ads_spending s2 
			   	WHERE (s1.date - s2.date) <= 30 AND
					  (s1.date - s2.date) >= 0 AND
			   		   s1.business = s2.business) AS moving_avg
		FROM ads_spending s1
	) t1
) t2;

# 5. LinkedIn job postings

In [ ]:
/* Facebook Connections

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE linkedin_job_postings (
	date VARCHAR,
	user_id FLOAT,
	receiver_id INTEGER,
	action VARCHAR 
); 


COPY linkedin_job_postings
FROM '<Path to product_sql folder>/product_sql/job_postings/linkedin_job_postings.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Hard
Duration: 15 Minutes

Problem:

On LinkedIn, companies companies can advertise job openings. The data team wants to categorize job posts 
as either ​active, expired, ​or​ repeats​.

> A post is active if it was published the past 30 days before 09/11/19 (current date). 
> A post is expired if it was published 30 days before 09/11/19.
> A post is a repeat if it was expired then active again.

Aaddress the following question: 

#1. Count the number of active and expired posts. Do not count repeats in the active-only or expired buckets.

*/


-- #1. Count the number of active and expired posts. Do not count repeats in the active-only or expired buckets.

-- Count the number of non-repeated active, expired and repeated posts.
-- Do not include repeats in the active-only bucket.

WITH post_snapshot AS (
-- For each post snapshot, tag whether the snapshot is active or expired
-- This label is required to remove repeats.
SELECT post_id, advertiser_id, timestamp,
	   CASE WHEN ('2019-09-11' - timestamp) <= 30 THEN 1
	        ELSE 0
	   END AS snapshot_active,
	   CASE WHEN ('2019-09-11' - timestamp) > 30 THEN 1
	        ELSE 0
	   END AS snapshot_expired
FROM job_postings
),
	post_status AS (
	-- Tag a post as active, expired, repeat given the logic: If the post
	-- contains 1 snapshot that's active and no snapshot expired, then it's
	-- active-only. If it contains no active and multiple expired snapshots,
	-- then it's expired. All other post instances are repeats. 
	SELECT post_id, advertiser_id,
		   CASE WHEN current_active = 1 AND number_of_expired = 0 THEN 'Active'
		        WHEN current_active = 0 AND number_of_expired > 0 THEN 'Expired'
		        ELSE 'Repeat'
		    END AS status
	FROM (
		-- For each post, count the number of snapshots that were active
		-- and expired. This will determine whether the post is active-only,
		-- expired or repeat.
		SELECT post_id, advertiser_id,
			   SUM(snapshot_active) AS current_active,
			   SUM(snapshot_expired) AS number_of_expired
		FROM post_snapshot
		GROUP BY 1, 2
	) t
)
-- Using post_status, count the number of instances.
SELECT status,
	   COUNT(*) AS counts
FROM post_status
GROUP BY 1;

# 6. Precision-Recall

In [ ]:
/* Precision-Recall

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE precision_recall (
	transaction_id VARCHAR,
	probability FLOAT,
	labels INTEGER,
	department VARCHAR 
); 


COPY precision_recall
FROM '<Path to product_sql folder>/product_sql/precision_recall/precision_recall.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Medium
Duration: 15 Minutes
    
df =sqlc.sql("""
WITH Prediction AS (
SELECT
CASE WHEN probability > 0.7 THEN 1 ELSE 0 END AS prediction,
labels
FROM metro_ca_analyst_tmp.di_v1
),
CorrectPrediction AS (
SELECT
prediction,
labels,
CASE WHEN prediction = 1 AND labels = 1 THEN 1 ELSE 0 END AS correct_prediction
FROM Prediction
)
SELECT
SUM(correct_prediction) * 1.0 / SUM(prediction) AS precision,
SUM(correct_prediction) * 1.0 / SUM(labels) AS recall
FROM CorrectPrediction
""")

df.show()

# df.write.saveAsTable('metro_ca_analyst_tmp.<>', mode='overwrite')

# 7. Revenue Analytics

In [ ]:
/* Revenue Analytics

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE revenue_analytics (
	company VARCHAR,
	year INTEGER,
	revenue FLOAT
); 


COPY revenue_analytics
FROM '<Path to product_sql folder>/product_sql/revenue_analytics/revenue_analytics.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Easy
Duration: 15 Minutes

Problem:

Given the company revenue data below, address two questions:

#1​. For each year, return the names of companies with the top 10th percentile 
revenue. Also return years and revenues.

#2​. Return the names of companies that grew their YoY revenue by at 
least 5%, consecutively every year.

*/


-- #1 For each year, return the names of companies with the top 10th percentile revenue. 
--    Also return years and revenues.

SELECT company, year, revenue
  FROM (
  	-- Within each year, rank companies based on their revenue in a descending order
	SELECT company, year, revenue,
		   ROW_NUMBER() OVER(PARTITION BY year ORDER BY revenue DESC) AS rank,
		   COUNT(*) OVER(PARTITION BY year) AS number_of_companies
	  FROM revenue_analytics
) t
WHERE (rank * 1.0  / number_of_companies) <= 0.10;


-- #2 Return the names of companies that grew their YoY revenue by at least 5%, consecutively every year.

WITH yoy_growth AS (
	-- Calculate YoY growth for each company by dividing the revenue of
	-- current year by last year. 
	SELECT company, year,
		   revenue * 1.0 / revenue_last_year yoy
	FROM (
		-- Shift the last year's revenue to the current year snapshot
		SELECT company, revenue, year,
			   LAG(revenue, 1) OVER(PARTITION BY company ORDER BY year) AS revenue_last_year
		FROM revenue_analytics
	) t
), 
growth_companies AS (
	-- For each company, count the number of consecutive years that achieved 5% growths and
	-- the number of years that the company existed. If the number_of_years - 1 == consecutive
	-- growths, then the company matches the criteria
	SELECT company,
		   SUM(CASE WHEN yoy >= 1.05 THEN 1 ELSE 0 END) AS consecutive_growths,
		   COUNT(*) AS number_of_years
	FROM yoy_growth
	GROUP BY 1
)
-- Return the list of companies that always achieved 5% growths in revenue.
SELECT company
FROM growth_companies
WHERE consecutive_growths = number_of_years - 1;

# 8. Statistical correlation

In [ ]:
/* statistical_correlation

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE stats  (
	x1 INTEGER,
	x2 INTEGER
); 


COPY stats
FROM '<Path to product_sql folder>/product_sql/statistical_correlation/stats.csv' 
DELIMITER ',' 
CSV HEADER;


/* 

Difficulty: Medium
Duration: 15 Minutes

Problem:

Calculate the statistical correlation of x1 and x2

*/


-- Correlation Coefficient Formula: COV(x1, x2) / [STD(x1) * STD(x2)]

WITH Mean AS (
	-- Mean
	SELECT
		x1,
		x2,
		AVG(x1) OVER() AS mean_x1,
		AVG(x2) OVER() AS mean_x2
	FROM stats
),
Variance AS (
	-- Variance
	SELECT
		AVG(POWER(x1 - mean_x1, 2)) AS var_x1,
		AVG(POWER(x2 - mean_x2, 2)) AS var_x2
	FROM Mean
), 
StandardDeviation AS (
	-- Standard deviation
	SELECT
		POWER(var_x1, 0.5) AS std_x1,
		POWER(var_x2, 0.5) AS std_x2
	FROM Variance
),
Covariance AS (
	-- Covariance
	SELECT 
		AVG((x1 - mean_x1) * (x2 - mean_x2)) AS cov_x1_x2
	FROM Mean
)
-- Correlation Coefficient
SELECT
	cov_x1_x2 / (std_x1 * std_x2) AS corr_x1_x2
FROM
	StandardDeviation, Covariance;

# 9. twitch_content_violations

In [ ]:
/* Twitch Content Violations

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE twitch_content_violations (
	date DATE,
	user_id VARCHAR,
	video INTEGER,
	sexual INTEGER,
	hateful INTEGER,
	spam INTEGER
); 


COPY twitch_content_violations
FROM '<Path to product_sql folder>/product_sql/twitch_content_violations/twitch_content_violations.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Hard
Duration: 30 Minutes

Problem:

The integrity team in Twitch, a video streaming platform for games, ensures that publishers 
follow a community guidelines that video content is not sexual, hateful or spammy. 
Use the violations table below to address three part questions. Address these two questions:

#1 - ​On a monthly basis, how many users publish at least one video that violates all 
three categories - sexual, hateful and spammy?

#2 - ​Currently, the integrity team doesn’t enforce banning a user unless the number of 
violations exceeds ten. A revision is proposed such that a user is banned if the number of 
violations accumulated ​exceeds three​. For each user, return two records:

	1. The first record shows the date, user_id and status “0” when a user 
	published a video for the first time.
	
	2. The last record shows the date, user_id and status “1” when a user 
	published a video for the last time before being banned.

For users who are not banned, only return the first record.
*/


-- #1 - On a monthly basis, how many users publish at least one video that violates all 
-- three categories - sexual, hateful and spammy?

-- The top-level query applies group by on year, month and counts rows that represent 
-- instances when users published videos that violated three rules.
SELECT year, month, COUNT(*)
FROM (
	-- The sub-query first filters on videos that violate all categories then removes 
	-- duplicate user_id's using DISTINCT on year,month,user_id
	SELECT DISTINCT
		EXTRACT(YEAR FROM date) AS year, 
	    EXTRACT(MONTH FROM date) AS month, 
	    user_id 
	FROM violations
	WHERE (sexual + hateful + spam) = 3 
) t
GROUP BY year, month;


-- #2 - Currently, the integrity team doesn’t enforce banning a user unless the number of violations exceeds ten. 
-- A revision is proposed such that a user is banned if the number of violations accumulated exceeds three. 
-- For each user, return two rows: the date of the first video and date when banned. Additionally, create an 
-- indicator column that shows status of whether a user signed up "0" or banned "1." The query should return
-- a table with user_id, status, date. Note that some users are not banned; hence, only one row is returned.


-- The first WITH clause calculates the cumulative violations per user across dates.
WITH cumulative_violation_table AS (
SELECT user_id,
               date,
	SUM(total_violations) OVER (PARTITION BY user_id ORDER BY date, video ASC) AS cumulative_violations
FROM (
SELECT user_id,
	   video,
	   date,
	   (sexual + hateful + spam) AS total_violations
FROM violations ) t1
),
-- The second WITH clause takes the sub-query that creates the status indicator below and 
-- sets the key dates, first and last videos, based on the date of the first video in each 
-- status per user.
status_minimum_date AS (
SELECT user_id, 
	   status, 
	   MIN(date) as date
FROM (
    -- The subquery creates a status indicator if the user is banned "1" or not "0" on a particular day.
	SELECT user_id, 
	 	   date,	
		   CASE WHEN cumulative_violations <= 3 THEN 0 ELSE  1 END AS status
FROM cumulative_violation_table ) t
GROUP BY user_id, status 
)
-- Final table that retrieves status and date per user.
SELECT * FROM status_minimum_date ORDER BY user_id, status;

# 10. Uber engagement

In [ ]:
/* Uber Engagement

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE uber_engagement_users (
	user_id VARCHAR,
	role VARCHAR,
	city VARCHAR,
	banned BOOLEAN 
); 

CREATE TABLE uber_engagement_trips (
	id VARCHAR,
	client_id VARCHAR,
	driver_id VARCHAR,
	status VARCHAR,
	request_date DATE
); 

COPY uber_engagement_users
FROM '<Path to product_sql folder>/product_sql/uber_engagement/uber_engagement_users.csv' 
DELIMITER ',' 
CSV HEADER;

COPY uber_engagement_trips
FROM '<Path to product_sql folder>/product_sql/uber_engagement/uber_engagement_trips.csv' 
DELIMITER ',' 
CSV HEADER;


/*

Difficulty: Hard
Duration: 30 Minutes

Problem:

There are two tables - the uber_engagement_users table contains user_id, role (i.e. driver, 
client) and city (i.e. NYC, SF, Berlin, Tokyo) and banned (i.e. T, F), and the
uber_engagement_trips table contains, id, client_id, driver_id, status (i.e. completed, 
cancelled_by_driver, cancelled_by_client), and request_date. 

Address the two questions:

#1. Between August 01, 2021 and August 12, 2021, what percentage of requests made by unbanned 
	clients each day were cancelled in each city? 
#2. Between August 01, 2021 and August 12, 2021, among SF and NYC, in each city, list top 
	three drivers by the number of completed trips.

*/


-- Solution #1
WITH UnbannedClients AS (
	-- This temp table filters on unbanned clients
	SELECT user_id, city
	FROM uber_engagement_users
	WHERE role = 'client'
	  AND banned = False
),
FilterTripsOnDates AS (
	-- This temp table filters the trip table given request dates
	SELECT 
		client_id AS user_id,
		status,
		request_date
	FROM uber_engagement_trips
	WHERE request_date BETWEEN '2021-08-01' AND '2021-08-12'
),
TripsByUnbannedClients AS (
	-- Table joins the trips to the client table
	SELECT user_id, city, status, request_date
	FROM FilterTripsOnDates JOIN UnbannedClients
		USING (user_id)
)
SELECT 
	-- By city and date, average the number of instances where the ride status 
	-- is not complete.
	city,
	request_date,
	AVG(CASE WHEN status != 'completed' THEN 1 ELSE 0 END) AS cancellation_rate 
FROM TripsByUnbannedClients
GROUP BY city, request_date
ORDER BY city, request_date;


-- Solution #2
WITH DriversInSelectCities AS (
	-- Filters on drivers in NYC and SF
	SELECT 
		user_id AS driver_id, 
		city
	FROM uber_engagement_users
	WHERE 
		city IN ('NYC', 'SF') AND
		role = 'driver'
),
DriverTrips AS (
	-- Filters on drivers who performed trips between dates provided.
	SELECT 
		driver_id,
		city,
		request_date,
		status
	FROM DriversInSelectCities
	JOIN uber_engagement_trips
		USING (driver_id)
	WHERE request_date BETWEEN '2021-08-01' AND '2021-08-12'
),
CompletedTrips AS (
	-- Count the number of completed trips per driver in each city
	SELECT
		city,
		driver_id,
		COUNT(*) AS trips_completed
	FROM DriverTrips 
	WHERE status = 'completed'
	GROUP BY city, driver_id
),
RankDrivers AS (
	-- Apply dense_rank as multiple drivers could have completed the same number of trips.
	SELECT city,
		trips_completed,
		DENSE_RANK() OVER(PARTITION BY city ORDER BY trips_completed) AS driver_rank
	FROM CompletedTrips
)
-- Filter on top three drivers.
SELECT city, driver_rank, trips_completed
FROM RankDrivers
WHERE driver_rank <= 3;

# 11. User Subscription

In [ ]:
/* User Subscription

Instruction:

Please run the CREATE TABLE and COPY commands below that will create the data tables
for this problem. Note that, in the COPY command, you will need to specify the directory 
where your data folder is located. 

For instance, if your SQL course folder is located in the home directory, then
set the path as /Home/product_sql/find_median/user_creation.csv.

Once your data tables are created, you are ready to solve the problem!

*/

CREATE TABLE user_subscription (
	user_id VARCHAR,
	timestamp INTEGER,
	action VARCHAR 
); 

COPY user_subscription
FROM '<Path to product_sql folder>/product_sql/user_subscription/user_subscription.csv' 
DELIMITER ',' 
CSV HEADER;

/*

Difficulty: Medium
Duration: 15 Minutes

Problem:

There is a table called user_action which contains user_id, timestamp and action (visit, subscribe, 
and cancel). A user can visit, subcribe, cancel multiple times. An example could be:

User A: Visit, Visit 
User B: Visit, Visit, Subscribe
User C: Visit, Visit, Subscribe, Cancel, Subscribe

Address the two questions:

#1. How long did it take for each user to subscribe after the first page visit?
#2. How many users cancelled then re-subscribed again? Only count the users who 
have remained subscribed after re-subscribing.

*/


-- Solution #1
WITH Visit AS (
	-- This temp table filters on users's first visit using row_number()
	SELECT user_id, timestamp
	FROM (	
		SELECT user_id, timestamp, 
			row_number() OVER(PARTITION BY user_id ORDER BY timestamp ASC) as row_number
		FROM user_subscription			
		WHERE action = 'visit'
	) subquery 
	WHERE row_number = 1
),
Subscribe AS (
	-- This temp table filters on users's first subscription using row_number() 
	-- Note that a user could cancel and re-subscribe. So, it's important to 
	-- filter on the first subscription record. 
	SELECT user_id, timestamp
	FROM (	
		SELECT user_id, timestamp,
			row_number() OVER(PARTITION BY user_id ORDER BY timestamp ASC) as row_number
		FROM user_subscription
		WHERE action = 'subscribe'	
	) subquery
	WHERE row_number = 1
)
-- Compute the difference in the timestamps between the first subscribe and first visit
-- timestamps.
SELECT Visit.user_id,
	   Subscribe.timestamp - Visit.timestamp AS delta
FROM visit JOIN Subscribe USING (user_id);


-- Solution #2
WITH LatestCancel AS (
	-- This temp table filters on users's last cancelled using row_number(). Similar to 
	-- subscription, a user can subscribe, cancel, re-subscribe, and re-cancel. So, it's
	-- important to select the last instance of churn using ORDER BY timestamp DESC.
	SELECT user_id, timestamp
	FROM (	
		SELECT user_id, timestamp,
			row_number() OVER(PARTITION BY user_id ORDER BY timestamp DESC) as row_number
		FROM user_subscription
		WHERE action = 'churn'	
	) subquery
	WHERE row_number = 1
),
LatestSubscribe AS (
	-- This temp table filters on users's last re-subscription using row_number(). Similar to 
	-- subscription, a user can subscribe, cancel, re-subscribe, and re-cancel. So, it's
	-- important to select the last instance of subscription using ORDER BY timestamp DESC.
	SELECT user_id, timestamp
	FROM (	
		SELECT user_id, timestamp,
			row_number() OVER(PARTITION BY user_id ORDER BY timestamp DESC) as row_number
		FROM user_subscription
		WHERE action = 'subscribe'	
	) subquery
	WHERE row_number = 1
)
-- JOIN LatestCancel and LatestSubscribe then filter on instances where the
-- timestamp in LatestSubscribe is greater than timestamp in LatestCancel.
-- Finally, count the records which returns the number of unique instances
-- where users re-subscribed and remained subscribed thus far.
SELECT COUNT(*)
FROM LatestCancel JOIN LatestSubscribe USING (user_id)
WHERE LatestSubscribe.timestamp > LatestCancel.timestamp;

